PROGRAMM CAR DECTECTION PREDICTIVE ANALYSIS & BIG DATA
TEAM 2
TEAMMITGLIEDER Tobias Madaj, Rickiel Eric Sympe Nguebong, Al Shah Aziz, Darwin Hutama Manggala Putra

In [3]:
import cv2
import pafy
import datetime
import numpy as np
import pandas as pd
from object_detection import ObjectDetection
import time
import math

# URL FROM YOUTUBE STREAM HERE
url = "https://www.youtube.com/watch?v=YByJ2h0T5JY"

camera_source = pafy.new(url).getbest()
capture = cv2.VideoCapture(camera_source.url)

#INIT THINGS FROM PROJECT CAN BE VERY IMPORTANT
od = ObjectDetection()

#VARIABles FOR CAR COUNT & CAR SPEED
carcount = 0
last_carspeed = 0

#DEFINE LISTS TO SAVE FOR LATER PANDAS DATAFRAME PROCESSING
#CARID = []
#CARSPEED = []
#CARCOLOUR = []
#DETECTIONTIME = []

#DEFINE AREA FOR DETECTION COORDINATES
area_detection = [(327,188),(549,171),(969,303),(563,413)]
area_speed = [(1213,369),(779, 561)]

#init Counting STUFF
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

#LOOP FOR OUTPUT CAPTURE VIDEO FOR LATER PROCESSING
while (True):

    # Capture frame-by-frame
    ret, current_frame = capture.read()

    count += 1
    if not ret:
        break

    center_points_cur_frame = []

    height, width, _ = current_frame.shape

    #BLEND TIME IN FRAME WITH COLOUR RIGHT HIGH CORNER INKLUSIVE FONT DEFINITION
    font_date = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
    font_carcount = cv2.FONT_ITALIC
    dt = str(datetime.datetime.now().strftime("%d.%m.%Y %H:%M:%S"))
    carcountstr = "Vehicle Detected: "
    lastspeedstr = "Last Speed: "


    #DEFINTION OF ROI (REGION OF INTEREST) IN PX FROM ORIGINAL FRAME
    #MUST BE RECTANCLE NOT RECTANGLE NOT WORK
    y1 = 153
    y2 = 425
    x1 = 298
    x2 = 976
    roi = current_frame[y1:y2, x1:x2]

    #DRAW AREA PERMANANTLY ON FRAME FOR ROI AND SPEED DETECTION WITH area_detection and area_speed VARIABLES
    #cv2.polylines(current_frame, [np.array(area_detection, np.int32)], True, (15, 220, 10), 4)
    cv2.polylines(current_frame, [np.array(area_speed, np.int32)], True, (212, 255, 127), 4)

    #Detect Objects on Frame
    (class_ids, scores,boxes) = od.detect(current_frame)
    for box in boxes:
        #print(box)
        (x,y,w,h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)

        center_points_cur_frame.append((cx,cy))
        #print("Frame N", count, " ", x,y,w,h)

        #cv2.circle(roi, pt, 5, (0,0,255), -1)
        cv2.rectangle(current_frame, (x,y), (x+w,y+h), (0,255,0), 2)

    # Only at the beginning we compare previous and current frame
    if count <= 2:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                if distance < 20:
                    tracking_objects[track_id] = pt
                    track_id += 1
    else:

        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()

        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                # Update IDs position
                if distance < 20:
                    tracking_objects[object_id] = pt
                    objects_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue

            #Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)
    # Add new IDs found
    for pt in center_points_cur_frame:
        tracking_objects[track_id] = pt
        track_id += 1

    for object_id, pt in tracking_objects.items():
        cv2.circle(roi, pt, 5, (0,0,255), -1)
        cv2.putText(roi, str(object_id), (pt[0], pt[1] - 7 ), 0, 1, (0, 0, 255), 2)


    current_frame = cv2.putText(current_frame, dt,(0, 25),font_date, 1,(0, 255, 255),4, cv2.LINE_8)
    current_frame = cv2.putText(current_frame, carcountstr+str(carcount),(0, 700),font_carcount, 1,(0, 0, 255),4, cv2.LINE_8)
    current_frame = cv2.putText(current_frame, lastspeedstr+str(last_carspeed)+" Km/h",(940, 25),font_carcount, 1,(0,0,128),4, cv2.LINE_8)

    cv2.imshow('Computer Vision CardetectionML-Frame', current_frame)
    #cv2.imshow("RANGE OF INTEREST", roi)
    #cv2.imshow("MASK",mask)
    cv2.resizeWindow('Computer Vision CardetectionML-Frame', 1280, 720)
    #cv2.waitKey(15)

    #Make a Copy of Points
    center_points_prev_frame = center_points_cur_frame.copy()

    #BREAK THE LOOP WITH KEYBOARD INTERRUPT
    if cv2.waitKey(18) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        capture.release()
        break

#HERE THE EXPORT TO DATAFRAME BEGINS
# VERY IMPORTANT

# make list to pandas series datatype

#se1 = pd.Series(year)
#se2 = pd.Series(votes)
#se4 = pd.Series(rating)

#df = pd.DataFrame(movie)

#CONCATANATION OF THE SERIES IMPORTANT STUFF
#extra = pd.concat([se1, se2,  se4], axis=1)
#final = pd.concat([df, extra], axis=1)

#Essential here is header=none
#header=None
#colum = ["CarID","CarSpeed", "CarColour", "Detectiontime"]

#final.to_csv("final.csv", header=colum, index_label="ID")

Stream OPENCV über Flask

In [7]:
from flask import Flask, render_template, Response
import cv2
import pafy
import datetime
import math
import numpy as np
import imutils
import pandas as pd
from object_detection import ObjectDetection
import time

app = Flask(__name__)

url = "https://www.youtube.com/watch?v=YByJ2h0T5JY"
od = ObjectDetection()
camera_source = pafy.new(url).getbest()
camera = cv2.VideoCapture(camera_source.url)

#DEFINE AREA FOR DETECTION COORDINATES
area_detection = [(327,188),(549,171),(969,303),(563,413)]
area_speed = [(1213,369),(779, 561)]

#VARIABles FOR CAR COUNT & CAR SPEED
carcount = 0
last_carspeed = 0

def gen_frames():  # generate frame by frame from LIVE STREAM
    while True:
        success, frame = camera.read()  # read the camera frame
        frame = imutils.resize(frame, width=1280,height=720 )

        # BLEND TIME IN FRAME WITH COLOUR RIGHT HIGH CORNER INKLUSIVE FONT DEFINITION
        font_date = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
        font_carcount = cv2.FONT_ITALIC
        dt = str(datetime.datetime.now().strftime("%d.%m.%Y %H:%M:%S"))
        carcountstr = "Vehicle Detected: "
        lastspeedstr = "Last Speed: "

        frame = cv2.putText(frame, dt, (0, 25), font_date, 1, (0, 255, 255), 4, cv2.LINE_8)
        frame = cv2.putText(frame, carcountstr + str(carcount), (0, 700), font_carcount, 1, (0, 0, 255),4, cv2.LINE_8)
        frame = cv2.putText(frame, lastspeedstr + str(last_carspeed) + " Km/h", (940, 25),font_carcount, 1, (0, 0, 128), 4, cv2.LINE_8)

        #DRAW AREA PERMANANTLY ON FRAME FOR ROI AND SPEED DETECTION WITH area_detection and area_speed VARIABLES
        cv2.polylines(frame, [np.array(area_detection, np.int32)], True, (15, 220, 10), 8)
        cv2.polylines(frame, [np.array(area_speed, np.int32)], True, (212, 255, 127), 8)

        #Detect Objects on Frame
        (class_ids, scores,boxes) = od.detect(frame)
        for box in boxes:
            #print(box)
            (x,y,w,h) = box
            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0),2)

        if not success:
            break
        else:
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')  # concat frame one by one and show result

@app.route('/video_feed')
def video_feed():
    #Video streaming route. Put this in the src attribute of an img tag
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

#RUN WEB-SERVER to see PANDAS DATAFRAME in BROWSER OVER FLASK WEB SERVER WITHOUT CSS
@app.route('/dataframe')
def hello():
    x = pd.read_csv('carexample.csv',index_col=0)
    return x.style.to_html()

@app.route('/')
def index():
    """Video streaming home page."""
    return render_template('index.html')

app.run(host='0.0.0.0', port=81)

127.0.0.1 - - [12/Jun/2022 11:00:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 11:00:16] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 11:00:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2022 11:00:49] "GET /video_feed HTTP/1.1" 200 -


Auswertung des Dataframes mittels Matplotlib

In [ ]:
import matplotlib as plt
# Abschnitt Programmauswertung

df1 = pd.read_csv('carexample.csv')

Das Auto mit der höchsten Geschwindigkeit

In [ ]:
df1.max()

Das Auto mit der niedrigster Geschwindigkeit

In [ ]:
df1.min()

Die durchschnitts Geschwindigkeit aller Autos

In [ ]:
df2 = df1['Speed'].mean()
df2